In [1]:
import dask.bag as db
import re
import json
import operator as op
from collections import Counter
import numpy as np

Подсчитать встречаемость слов в отзывах с оценкой 2 и отзывах с оценкой 5

In [2]:
jbag = db.read_text('datasets/Toys_and_Games_5.json')

In [15]:
res = jbag.map(json.loads).filter(lambda x: x['overall'] == 2).pluck('reviewText').compute()

In [11]:
jbag_5 = db.from_sequence(res)

In [13]:
res_5 = jbag_5.map(count_json_new).fold(op.add).compute()

In [16]:
jbag_2 = db.from_sequence(res)

In [19]:
res_2 = jbag_2.map(count_json_new).fold(op.add).compute()

In [21]:
res_2 is res_5

False

In [55]:
sumval = sum(res_2.values())
for key in list(res_2.keys()):
    res_2[key] = res_2[key] / sumval

In [57]:
sumval = sum(res_5.values())
for key in list(res_5.keys()):
    res_5[key] = res_5[key] / sumval

In [69]:
sett = (res_5 - res_2 | res_2 - res_5)
sett.most_common()[0:49]

[('the', 0.007737090413038744),
 ('and', 0.006365167372917483),
 ('not', 0.004757842958095125),
 ('t', 0.00421529926290076),
 ('it', 0.003881027068411768),
 ('great', 0.0032553144559451567),
 ('for', 0.0030200010614521867),
 ('but', 0.002845794104093356),
 ('i', 0.0027930827911400516),
 ('with', 0.0024843840651721273),
 ('my', 0.002465918675196535),
 ('loves', 0.0024004880114209945),
 ('he', 0.002301309566641568),
 ('love', 0.0022612546764922948),
 ('a', 0.0020417644520528133),
 ('she', 0.0019963134857299164),
 ('her', 0.0018047425884415783),
 ('would', 0.0017677059047116286),
 ('is', 0.0017106393742170323),
 ('this', 0.001695649388336263),
 ('was', 0.001671669641977058),
 ('that', 0.0015447855814060552),
 ('old', 0.0014785794222540345),
 ('has', 0.00145476878613904),
 ('fun', 0.0013730750992410595),
 ('year', 0.0013046926267522001),
 ('to', 0.0012623182861889216),
 ('little', 0.0012612272116392692),
 ('off', 0.001217822394812067),
 ('can', 0.0012048845856797471),
 ('his', 0.0012026625

In [64]:
type(sett)

collections.Counter

In [4]:
def count_json_new(line):
    reslist = re.findall('\w+', line.lower())
    return Counter(reslist)

Подсчитать слова в отзывах с оценкой 5

In [16]:
def count_json(line):
    if (line['overall'] == 5.0):
        reslist = re.findall('\w+', line['reviewText'].lower())
        return Counter(reslist)
    else: return Counter()

In [7]:
jbag = db.read_text('datasets/Toys_and_Games_5.json')

In [8]:
res = jbag.map(json.loads).compute()

In [10]:
type(res)

list

In [11]:
resbag = db.from_sequence(res)

In [17]:
resbag.map(count_json).fold(op.add).compute()

Counter({'i': 171460,
         'like': 30117,
         'the': 429965,
         'item': 3189,
         'pricing': 105,
         'my': 103440,
         'granddaughter': 4149,
         'wanted': 4075,
         'to': 227774,
         'mark': 252,
         'on': 63687,
         'it': 232686,
         'but': 56081,
         'just': 28772,
         'for': 142466,
         'letters': 2100,
         'both': 7213,
         'sides': 839,
         'are': 70091,
         'magnetic': 599,
         'a': 231420,
         'real': 3797,
         'plus': 2488,
         'when': 27982,
         'you': 70254,
         're': 6251,
         'entertaining': 874,
         'more': 23850,
         'than': 15042,
         'one': 42627,
         'child': 10729,
         'four': 3517,
         'year': 30564,
         'old': 39003,
         'can': 44560,
         'find': 6576,
         'words': 1822,
         'while': 8522,
         'two': 13272,
         'pictures': 2333,
         'spell': 277,
         'bought': 16

Подсчитать топ 5 дат, за которые оставлено больше всего отзывов

In [2]:
jbag_days = db.read_text('datasets/Toys_and_Games_5.json')

In [62]:
dates = jbag_days.map(json.loads).pluck('reviewTime')
#это, возможно, довольно дорого с точки зрения ресурсов
res = dates.frequencies().topk(5, lambda x: x[1])

In [63]:
res.compute()

[('12 26, 2013', 887),
 ('12 28, 2012', 878),
 ('12 27, 2013', 842),
 ('12 30, 2013', 827),
 ('12 26, 2012', 819)]

Вывести в порядке убывания популярности все отзывы, встретившиеся более 1 раза

In [3]:
summaries = jbag_days.map(json.loads).pluck('summary')
summaries = summaries.frequencies(sort = True).filter(lambda el: el[1] > 1)

In [4]:
summaries.compute()

[('Cute', 558),
 ('Great', 532),
 ('Five Stars', 509),
 ('Fun', 498),
 ('Great toy', 424),
 ('Great!', 388),
 ('Love it', 373),
 ('Fun!', 333),
 ('Love it!', 319),
 ('Great toy!', 309),
 ('Awesome', 287),
 ('cute', 248),
 ('Awesome!', 247),
 ('Perfect', 237),
 ('great', 233),
 ('Cute!', 227),
 ('love it', 197),
 ('Very cute', 194),
 ('Great Toy', 194),
 ('Nice', 192),
 ('Fun toy', 184),
 ('Adorable', 178),
 ('Great game', 177),
 ('fun', 174),
 ('Lots of fun', 163),
 ('Adorable!', 160),
 ('Great product', 155),
 ('Gift', 149),
 ('great toy', 148),
 ('Perfect!', 146),
 ('Great gift', 145),
 ('Great Game', 136),
 ('Good', 132),
 ('So cute!', 125),
 ('So much fun!', 124),
 ('Great fun', 124),
 ('Excellent', 115),
 ('Fun game', 113),
 ('Disappointed', 111),
 ('Great Toy!', 105),
 ('Great Product', 104),
 ('ravage 24/7', 103),
 ('Love', 102),
 ('Great Gift', 102),
 ('Kids love it', 102),
 ('Great game!', 101),
 ('gift', 100),
 ('Cute toy', 99),
 ('Great set', 97),
 ('Lots of fun!', 96),
 ('S